In [ ]:
def identify_sengene_v3(sencell_dict,gene_cell,edge_index_selfloop,attention_scores,
                        sen_gene_ls,ct_sencell_indexs):
    attention_scores=attention_scores.detach().to('cpu')
    edge_index_selfloop=edge_index_selfloop.detach().to('cpu')

    num_genes = gene_cell.shape[0]
    num_cells = gene_cell.shape[1]
    cell_mask_template = torch.zeros(num_genes + num_cells, dtype=torch.bool)

    gene_scores=[]
    for gene_index in range(num_genes):
        if gene_index %1000==0:
            print(gene_index)
        if gene_index in sen_gene_ls or gene_index in high_gene_indexs:
            # 保存一个基因在8个ct的snc上的score
            ct_scores=[]
            for key,value in ct_sencell_indexs.items():
                cell_index=value
                cell_index=torch.tensor(cell_index)
                
                cell_mask = cell_mask_template.clone()
                cell_mask[cell_index] = True
                
                connected_cells=edge_index_selfloop[0][edge_index_selfloop[1] == gene_index]
                
                if len(connected_cells[cell_mask[connected_cells]])==0:
                    ct_scores.append(torch.tensor(0))
                    # print(f'no sencell in this gene {gene_index}')
                else:
                    # print('caculate attention')
                    tmp=attention_scores[edge_index_selfloop[1] == gene_index]
                    attention_edge=torch.sum(tmp[cell_mask[connected_cells]],axis=1)
                    #cell_mask_normal=~cell_mask
                    #attention_edge_normal=torch.sum(attention_scores[edge_index_selfloop[1] == gene_index][cell_mask_normal[connected_cells]],axis=1)
                    #print(cell_mask,cell_mask_normal)
                    # print(attention_edge,attention_edge_normal)
                    attention_s=torch.mean(attention_edge)
                    #attention_normal=torch.mean(attention_edge_normal)
                    
                    #attention_s=attention_s-attention_normal
                    # print(attention_s)
                    ct_scores.append(torch.tensor(0.1))

            # 只计算老化基因还是对所有基因
            # if gene_index in sen_gene_ls:
            #     # print("sng:",attention_s)
            #     gene_scores.append(ct_scores)
            gene_scores.append(ct_scores)
        else:
            gene_scores.append([torch.tensor(0)]*8)
        
    return gene_scores

gene_scores=identify_sengene_v3(
        sencell_dict,new_data.X.T,edge_index_selfloop,attention_scores,sen_gene_ls,ct_sencell_indexs)

gene_scores=torch.tensor(gene_scores)
filtered_gene_scores=[]
filtered_gene_names=[]
for i in range(gene_scores.shape[0]):
    if torch.sum(gene_scores[i])!=0:
        filtered_gene_names.append(new_data.var.index[i])
        filtered_gene_scores.append(gene_scores[i])
        
filtered_gene_scores=torch.stack(filtered_gene_scores)
def calculate_outliers(scores):
    counts=0
    snc_index=[]

    scores_ls=[]
    
    Q1 = np.percentile(scores, 25)
    Q3 = np.percentile(scores, 75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    for i,score in enumerate(scores): 
        if score > upper_bound:
            counts+=1
            snc_index.append(i)
            scores_ls.append(score)
    
    return counts,snc_index,scores_ls

gene_deepsas={}

gene_names=list(new_data.var_names)
gene_names_sub=[gene_names[i] for i in sen_gene_ls]

for i,(key,value) in enumerate(ct_sencell_indexs.items()):
    scores=filtered_gene_scores[:,i]
    counts,sng_index,scores_ls=calculate_outliers(scores)
    # 计算在每个cluster里面有多少离群gene
    # 就是table1
    print(key,counts)
    sng_names=[[filtered_gene_names[i],float(scores_ls[j])] for j,i in enumerate(sng_index)]
    gene_deepsas[key]=sng_names


rows = []
for cell_type, genes in gene_deepsas.items():
    for gene, score in genes:
        rows.append([cell_type, gene, score])

df1 = pd.DataFrame(rows, columns=['Cell Type', 'Gene Name', 'Score'])

df1.to_csv("/bmbl_data/chenghao/sencell/SnGs_data2/table1.csv",index=0)
filtered_degs={}
for key,value in deg_results_full.items():
    filtered_genes=value[(value['logFC']>=0.25)]['gene'].tolist()
    #print(key, filtered_genes)
    filtered_degs[key]=filtered_genes
    print(key,len(filtered_genes))
    
results=[]
for key,value in deg_results_full.items():
    deg_results_full[key]['cell type']=key
    results.append(deg_results_full[key])

pd.concat(results).to_csv("./1.csv")
gene_deepsas_names={}
for key,value in gene_deepsas.items():
    gene_names=[line[0] for line in value]
    gene_deepsas_names[key]=gene_names
gene_deepsas_names

# For table 2
overlap_genes={}
for ct in selected_ct:
    ls1=set(gene_deepsas_names[ct])
    ls2=set(filtered_degs[ct])
    ls3=ls1.intersection(ls2)
    print(ct, len(ls3))
    overlap_genes[ct]=list(ls3)    

table2=[]
for ct in selected_ct:
    t=deg_results_full[ct]
    t=t[t['gene'].isin(overlap_genes[ct])]
    t['cell type']=ct
    table2.append(t)

table2=pd.concat(table2)

table2.to_csv("/bmbl_data/chenghao/sencell/SnGs_data2/table2.csv",index=0)
# The genes in table 3 is unique gene only occur in one cell type, I use excel to manually select genes in table3
# No code to generate table3